# **Delivery Dron Analysis**

# Описание проекта
Необходимо проанализировать данные иностранной интернет компании “Delivery Dron”, занимающейся продажей дронов доставщиков и работающей в четырех регионах c мая 2019 года.
## Задачи:

*   сделать отчет о вложенных средствах
*   построить планы на следующий год
*   выяснить  в какой регион и какой рекламный канал инвестировать больше денег на привлечение клиентов в 2020 году



Необходимо проанализировать имеющиеся данные и ответить на вопрос заказчика: в какой регион и какой рекламный канал инвестировать больше денег на привлечение клиентов в 2020 году?




# Шаг 0: Подготовка

## 0.1: Импорт библиотек

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

## 0.2: Константы и настройки

## 0.3: Загрузка данных

In [3]:
df = pd.read_csv("data.csv")
df.head()

,User Id,Region,Device,Channel,Session Start,Session End,SessionDurationSec,Session Date,Month,Day,Hour of day,Order Dt,Revenue,Payment Type,Promo code
0,529697267522,United States,iPhone,социальные сети,2019-05-01 00:06:40,2019-05-01 00:07:06,26.0,2019-05-01,5,3,0,2019-05-01 00:06:40,9999.0,Mobile payments,0.0
1,601292388085,United States,PC,organic,2019-05-01 06:56:16,2019-05-01 07:09:18,782.0,2019-05-01,5,3,7,NaN,NaN,NaN,NaN
2,852898876338,United States,Mac,социальные сети,2019-05-01 04:30:45,2019-05-01 04:34:56,251.0,2019-05-01,5,3,4,NaN,NaN,NaN,NaN
3,998513020664,United States,iPhone,социальные сети,2019-05-01 18:53:42,2019-05-01 18:57:35,233.0,2019-05-01,5,3,18,NaN,NaN,NaN,NaN
4,240702200943,United States,Mac,социальные сети,2019-05-02 14:04:32,2019-05-02 14:09:51,319.0,2019-05-02,5,4,14,NaN,NaN,NaN,NaN


## 0.4: Пользовательские функции

In [4]:
def get_ohe(data: pd.DataFrame, categories: list):
    """
    Функция, которая переводит номинальные шкалы в бинарные методом OneHotEncoder.
    
    :param data: dataframe
    :param categories: список названий колонн
    :return: новый dataframe
    """
    ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
    ohe.fit(data[categories])
  
    temp_df = pd.DataFrame(data=ohe.transform(data[categories]), columns=ohe.get_feature_names_out())
    data = pd.concat([data.reset_index(drop=True), temp_df], axis=1)
    data = data.drop(columns=categories, axis=1)
    return data

## **Шаг 1: Исследовательский анализ данных и предобработка**

In [5]:
df.columns = df.columns.str.lower().str.replace(" ", "_")